## Аналіз A/B-тестів

Ви - аналітик даних в ІТ-компанії і до вас надійшла задача проаналізувати дані A/B тесту в популярній [грі Cookie Cats](https://www.facebook.com/cookiecatsgame). Це - гра-головоломка в стилі «з’єднай три», де гравець повинен з’єднати плитки одного кольору, щоб очистити дошку та виграти рівень. На дошці також зображені співаючі котики :)

Під час проходження гри гравці стикаються з воротами, які змушують їх чекати деякий час, перш ніж вони зможуть прогресувати або зробити покупку в додатку.

У цьому блоці завдань ми проаналізуємо результати A/B тесту, коли перші ворота в Cookie Cats було переміщено з рівня 30 на рівень 40. Зокрема, ми хочемо зрозуміти, як це вплинуло на утримання (retention) гравців. Тобто хочемо зрозуміти, чи переміщення воріт на 10 рівнів пізніше якимось чином вплинуло на те, що користувачі перестають грати в гру раніше чи пізніше з точки зору кількості їх днів з моменту встановлення гри.

Будемо працювати з даними з файлу `cookie_cats.csv`. Колонки в даних наступні:

- `userid` - унікальний номер, який ідентифікує кожного гравця.
- `version` - чи потрапив гравець в контрольну групу (gate_30 - ворота на 30 рівні) чи тестову групу (gate_40 - ворота на 40 рівні).
- `sum_gamerounds` - кількість ігрових раундів, зіграних гравцем протягом першого тижня після встановлення
- `retention_1` - чи через 1 день після встановлення гравець повернувся і почав грати?
- `retention_7` - чи через 7 днів після встановлення гравець повернувся і почав грати?

Коли гравець встановлював гру, його випадковим чином призначали до групи gate_30 або gate_40.

1. Для початку, уявімо, що ми тільки плануємо проведення зазначеного А/B-тесту і хочемо зрозуміти, дані про скількох користувачів нам треба зібрати, аби досягнути відчутного ефекту. Відчутним ефектом ми вважатимемо збільшення утримання на 1% після внесення зміни. Обчисліть, скільки користувачів сумарно нам треба аби досягнути такого ефекту, якщо продакт менеджер нам повідомив, що базове утримання є 19%.

In [ ]:
Оскільки ми не знаємо, чи вплине перенесення воріт до 40 рівня на утримання гравців,
ми виберемо двосторонній тест:

H0:p=p0 
Ha:p≠p0 

де  p  і  p0  позначають коефіцієнт утримання гравців відповідно до наявності воріт на сороковому
та тридцятому рівнях.

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.api as sms
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil

%matplotlib inline

In [2]:
effect_size = sms.proportion_effectsize(0.20, 0.19)    # Розрахунок розміру ефекту на основі наших очікуваних показників
effect_size

0.025241594409087353

In [23]:
required_n = sms.NormalIndPower().solve_power(
    effect_size,
    power=0.8,
    alpha=0.05,
    ratio=1
    )                                                  # Розрахунок необхідного розміру вибірки

required_n = ceil(required_n)                          # Округлення до наступного цілого числа

print(required_n)

24638


2. Зчитайте дані АВ тесту у змінну `df` та виведіть середнє значення показника показник `retention_7` (утримання на 7 день) по версіям гри. Сформулюйте гіпотезу: яка версія дає краще утримання через 7 днів після встановлення гри?

In [24]:
df = pd.read_csv('cookie_cats.csv')
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [25]:
session_counts = df['userid'].value_counts(ascending=False)
multi_users = session_counts[session_counts > 1].count()

print(f'Є {multi_users} користувачів, які зустрічаються кілька разів у наборі даних.')

Є 0 користувачів, які зустрічаються кілька разів у наборі даних.


In [26]:
gate30_sample = df[df['version'] == 'gate_30'].sample(n=required_n, random_state=22)
gate40_sample = df[df['version'] == 'gate_40'].sample(n=required_n, random_state=22)

ab_test = pd.concat([gate30_sample, gate40_sample], axis=0)
ab_test.reset_index(drop=True, inplace=True)

In [27]:
ab_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49276 entries, 0 to 49275
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          49276 non-null  int64 
 1   version         49276 non-null  object
 2   sum_gamerounds  49276 non-null  int64 
 3   retention_1     49276 non-null  bool  
 4   retention_7     49276 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 1.2+ MB


In [28]:
retention_rates = ab_test.groupby('version')['retention_7']
retention_rates = retention_rates.agg(['mean', 'std', stats.sem])
retention_rates.columns = ['retention_rate', 'std_deviation', 'std_error']


retention_rates.style.format('{:.5f}')

,retention_rate,std_deviation,std_error
version,,,
gate_30,0.18914,0.39163,0.00249
gate_40,0.17798,0.38250,0.00244


In [ ]:
Судячи зі статистики вище,  версія gate_30 дає краще утримання через 7 днів після встановлення 
гри.


3. Перевірте з допомогою пасуючого варіанту z-тесту, чи дає якась з версій гри кращий показник `retention_7` на рівні значущості 0.05. Обчисліть також довірчі інтервали для варіантів до переміщення воріт і після. Виведіть результат у форматі:

    ```
    z statistic: ...
    p-value: ...
    Довірчий інтервал 95% для групи control: [..., ...]
    Довірчий інтервал 95% для групи treatment: [..., ...]
    ```

    де замість `...` - обчислені значення.
    
    В якості висновку дайте відповідь на два питання:  

      1. Чи є статистична значущою різниця між поведінкою користувачів у різних версіях гри?   
      2. Чи перетинаються довірчі інтервали утримання користувачів з різних версій гри? Про що це каже?  


In [16]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [18]:
gate_30_results = ab_test[ab_test['version'] == 'gate_30']['retention_7']
gate_40_results = ab_test[ab_test['version'] == 'gate_40']['retention_7']

In [22]:
n_30 = gate_30_results.count()
n_40 = gate_40_results.count()
successes = [gate_30_results.sum(), gate_40_results.sum()]
nobs = [n_30, n_40]
successes

[3623, 3455]

In [29]:
z_stat, pval = proportions_ztest(successes, nobs=nobs) 
(lower_30, lower_40), (upper_30, upper_40) = proportion_confint(successes, nobs=nobs, alpha=0.05)

print(f'z statistic: {z_stat:.2f}')
print(f'p-value: {pval:.3f}')
print(f'Довірчий інтервал 95% для групи gate_30: [{lower_30:.3f}, {upper_30:.3f}]')
print(f'Довірчий інтервал 95% для групи gate_40: [{lower_40:.3f}, {upper_40:.3f}]')


z statistic: 2.21
p-value: 0.027
Довірчий інтервал 95% для групи gate_30: [0.181, 0.192]
Довірчий інтервал 95% для групи gate_40: [0.173, 0.183]


In [ ]:

Оскільки наше  p-значення=0.027 нище за наше  α  = 0.05, ми маємо відхилити нульову 
гіпотезу  H0 , що означає, що перенесення воріт впливає на коефіцієнт утримання (але не в позитивний бік).

Також, якщо ми подивимося довірчий інтервал для групи gate_40 ([0,173, 0,183], тобто 17,3-18,3%), 
ми помітимо, що:

він не включає наше базове значення коефіцієнта утримання 19%.
Він не включає наше цільове значення в 20% (зростання на 1%, до якого ми прагнули).
Це означає, що перенесення воріт погіршило результат. І потрібно або залишити їх на 30му рівні 
або придумати щось інше.

4. Виконайте тест Хі-квадрат на рівні значущості 5% аби визначити, чи є залежність між версією гри та утриманням гравця на 7ий день після реєстрації.

    - Напишіть, як для цього тесту будуть сформульовані гіпотези.
    - Проведіть обчислення, виведіть p-значення і напишіть висновок за результатами тесту.


In [ ]:
H0:p=p0 перенесення воріт до 40 рівня не вплине на утримання гравців
Ha:p≠p0 перенесення воріт до 40 рівня матиме вплив на утримання гравців


In [30]:
import scipy.stats as stats

ab_test.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,7540471,gate_30,45,True,False
1,3589138,gate_30,21,True,False
2,3177668,gate_30,14,True,False
3,2133884,gate_30,26,False,False
4,492763,gate_30,39,True,True


In [31]:
pd.crosstab(ab_test['version'], ab_test['retention_7'])

retention_7,False,True
version,,
gate_30,19978,4660
gate_40,20253,4385


In [32]:
crosstab = pd.crosstab(ab_test['version'], ab_test['retention_7'])

In [33]:
chi2, p, dof, expected = stats.chi2_contingency(crosstab)

print(f"χ² = {chi2:.3f}")
print(f"p-value = {p:.5f}")
print(f"Ступені свободи = {dof}")
print("Очікувані частоти:\n", expected)

χ² = 10.166
p-value = 0.00143
Ступені свободи = 1
Очікувані частоти:
 [[20115.5  4522.5]
 [20115.5  4522.5]]


In [ ]:
 p-значення=0.00143 нище за наше  α  = 0.05, ми маємо відхилити нульову 
гіпотезу  H0 , що означає, що перенесення воріт впливає на коефіцієнт утримання